<a href="https://colab.research.google.com/github/avivyossef29/mixing-mechs-nlp/blob/nitzan%2Fbinding-tests/XGLM_7_5b_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install core libs (No IPython upgrade)
!pip install -q bitsandbytes transformers accelerate ipywidgets peft
!pip install -q "protobuf<6.0.0" "tensorboard<2.20.0" sentencepiece tqdm tabulate

# Re-clone your repo
!git clone https://github.com/NitzanZacharia/mixing-mechs-nlp.git

In [ ]:
import sys
import importlib
from types import ModuleType

# This "tricks" Python into thinking 'imp' still exists
if 'imp' not in sys.modules:
    imp = ModuleType('imp')
    imp.reload = importlib.reload
    sys.modules['imp'] = imp
    print("✅ 'imp' module shimmed successfully. Autoreload should now work.")

In [ ]:
!pip install -q pyvene

In [ ]:
!pip install -q nnsight

In [ ]:
!pip install -U traitlets

In [ ]:
# Setup (run once)
%load_ext autoreload
%autoreload 2

import os
import sys

# Move to repo and add to path
repo_dir = '/content/mixing-mechs-nlp'
os.chdir(repo_dir)
sys.path.append(repo_dir)
sys.path.append(os.path.join(repo_dir, "CausalAbstraction"))

# Your logging and imports
import logging
logging.getLogger("transformers.configuration_utils").setLevel(logging.ERROR)
import transformers
transformers.logging.set_verbosity_error()

import torch
from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

from grammar.schemas import SCHEMA_BOXES
from grammar.task_to_causal_model import multi_order_multi_schema_task_to_lookbacks_generic_causal_model
from training import get_counterfactual_datasets, sample_answerable_question_template

print("🚀 Script initialized successfully!")

In [ ]:
# Configuration
model_id = "facebook/xglm-7.5B"
schema = SCHEMA_BOXES
num_instances = 20  # Number of binding groups (n=20 as in paper)
num_samples = 100   # Number of test samples (use 3000 for full test)
cat_indices_to_query = [0]  # Query by Object
cat_to_query = 1            # Answer is Box


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from transformers import BitsAndBytesConfig
import torch

# 1. Configure 4-bit quantization (Essential for T4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16  # GPU likes float16, not float32
)

print(f"[+] Loading model: {model_id} on GPU...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",              # "auto" tells it to use the GPU (CUDA:0)
    low_cpu_mem_usage=True,         # Prevents crashing system RAM
    attn_implementation="eager"
)

print(f"✅ Model loaded on: {model.device}")


In [ ]:
print(f"    Model has {model.config.num_layers} layers")

In [ ]:
# Generate Test Dataset
print(f"[+] Generating test dataset with {num_instances} binding groups...")
causal_model = multi_order_multi_schema_task_to_lookbacks_generic_causal_model(
    [schema], num_instances, num_fillers_per_item=0, fillers=False
)
causal_models = {schema.name: causal_model}

train_ds, test_ds, _ = get_counterfactual_datasets(
    None,  # No pipeline for filtering
    [schema],
    num_samples=num_samples,
    num_instances=num_instances,
    cat_indices_to_query=cat_indices_to_query,
    answer_cat_id=cat_to_query,
    do_assert=False,
    do_filter=False,
    causal_models=causal_models,
    sample_an_answerable_question=sample_answerable_question_template,
)

train = train_ds[schema.name][schema.name]
print(f"    Generated {len(train)} samples")


In [ ]:
import re

def format_xglm_binding_prompt(prompt: str) -> str:
    # Extract the core logical statement
    context = prompt.split(' Respond')[0]
    try:
        question_part = prompt.split("nothing else:")[1].split("Box Answer:")[0].strip()
    except IndexError:
        question_part = "Which box is the object in?"

    # Few-Shot with explicit Language Tags (en:)
    # This keeps XGLM grounded in English
    few_shot = (
        "en: The scarf is in Box X, the basketball is in Box Y. "
        "Question: Which box is the scarf in? Answer: Box X\n"
        "###\n"
        "en: The cookie is in Box J, the keyboard is in Box K. "
        "Question: Which box is the keyboard in? Answer: Box K\n"
        "###\n"
    )

    return f"{few_shot}en: {context} Question: {question_part} Answer: Box"

def naive_box_checker(response: str, expected: str) -> bool:
    """
    Checks if the expected letter appears as a standalone word.
    Does NOT handle the 'BoxH' (no space) edge case.
    """
    # Find all words (sequences of alphanumeric characters)
    words = re.findall(r'\w+', response.upper())

    # Check if the expected letter is in that list
    return expected.strip().upper() in words

def generate_response(text, max_new_tokens=5):
    # Always add the BOS token for XGLM
    inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True).to(model.device)

    # XGLM often generates "</s>" (token 2) to end a sentence
    stop_strings = ["\n", "###", "</s>"]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded_output = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

    # Clean up output
    clean_output = decoded_output
    for stop_word in stop_strings:
        if stop_word in clean_output:
            clean_output = clean_output.split(stop_word)[0]

    return clean_output.strip()

In [ ]:

#73 so far
def format_xglm_binding_prompt3(raw_context: str, question_object: str) -> str:
    # 1. Start with the BOS token
    bos = "<s>"

    # 2. Stronger Few-Shot (Using 3-4 objects helps the model handle longer lists)
    few_shot = (
        "The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. "
        "Question: Which box is the ocarina in? Answer: Box V. ### "
        "The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. "
        "Question: Which box is the sextant in? Answer: Box X. ### "
    )

    # 3. Clean the raw_context
    # This removes the "Respond in one word..." part that is already inside the data
    # It keeps only the sentences describing where objects are.
    clean_context = raw_context.split("Respond in one word")[0].strip()

    # 4. Final Formatting
    # We ensure a space after the context and a clean question at the end.
    test_context = f"{clean_context} "
    question = f"Question: Which box is the {question_object} in? Answer: Box"

    return f"{bos}{few_shot}{test_context}{question}"


In [ ]:
#also 73
def format_xglm_binding_prompt2(raw_context: str, question_object: str) -> str:
    # 1. REMOVE MANUAL BOS (The tokenizer handles this)
    # Removing the manual "<s>" prevents the double-start-token confusion.

    # 2. DIVERSE FEW-SHOTS (Small Adjustment)
    # Example 1: Queries the FIRST item (Ocarina) -> Teaches Primacy
    # Example 2: Queries the LAST item (Anchor) -> Teaches Recency
    # This balance helps the model cover the whole context window.
    few_shot = (
        "The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. "
        "Question: Which box is the ocarina in? Answer: Box V. ### "

        "The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. "
        "Question: Which box is the plectrum in? Answer: Box Z. ### "
    )

    # 3. Clean Context (Kept exactly as you had it)
    clean_context = raw_context.split("Respond in one word")[0].strip()

    # 4. Final Formatting
    # We add a newline before the Question to help separation.
    test_context = f"{clean_context} "
    question = f"Question: Which box is the {question_object} in? Answer: Box"

    return f"{few_shot}{test_context}{question}"

# Keep your generation function exactly the same
def generate_response(text, max_new_tokens=2):
    # Ensure add_special_tokens=True is ON (default)
    inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True).to(model.device)

    stop_strings = ["\n", "###", "</s>"]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded_output = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

    clean_output = decoded_output
    for stop_word in stop_strings:
        if stop_word in clean_output:
            clean_output = clean_output.split(stop_word)[0]

    return clean_output.strip()

In [ ]:
# Test Single Sample (for debugging)
sample = train[0]
raw_input = sample["input"]["raw_input"]
queried_object = sample["input"].get("Object.0.Query", None)

print("Raw input")
print(raw_input)
print(f"\nQueried object: {queried_object}")

# Get expected answer
forward_res = causal_model.run_forward(sample["input"])
expected = forward_res.get("answer", "")
if isinstance(expected, dict):
    expected = max(expected, key=lambda k: expected[k])
print(f"Expected answer: {expected}")

# Format and generate
prompt = format_xglm_binding_prompt2(raw_input, queried_object)
print(f"\nFormatted prompt")
print(prompt)

response = generate_response(prompt)
print(f"\nModel response: '{response}'")
print(f"Correct (strict): {naive_box_checker(response, expected)}")
print(f"Correct (original): {schema.checker(response, expected)}")


In [ ]:
from tqdm import tqdm
# Run Full Evaluation
print(f"[+] Evaluating {len(train)} samples...")

correct = 0
total = 0
position_correct = {}
position_total = {}
results = []

for i in tqdm(range(len(train))):
    sample = train[i]
    raw_input = sample["input"]["raw_input"]
    queried_object = sample["input"].get("Object.0.Query", None)

    # Get expected answer
    forward_res = causal_model.run_forward(sample["input"])
    expected = forward_res.get("answer", "")
    if isinstance(expected, dict):
        expected = max(expected, key=lambda k: expected[k])

    # Get query position
    query_position = sample["input"].get("metadata", {}).get("src_positional_index", -1)

    # Format and generate
    prompt = format_xglm_binding_prompt2(raw_input, queried_object)
    response = generate_response(prompt)

    # Check correctness using STRICT checker
    is_correct = naive_box_checker(response, expected)
    if is_correct:
        correct += 1
    total += 1

    # Track per-position accuracy
    if query_position >= 0:
        if query_position not in position_correct:
            position_correct[query_position] = 0
            position_total[query_position] = 0
        if is_correct:
            position_correct[query_position] += 1
        position_total[query_position] += 1

    results.append({
        "expected": expected,
        "response": response,
        "correct": is_correct,
        "position": query_position,
        "queried_object": queried_object,
        "full_query": prompt,
    })

# Print results
print(f"\n{'='*50}")
print(f"RESULTS: XGLM-7.5B on Binding Task (n={num_instances})")
print(f"{'='*50}")
print(f"Overall Accuracy: {correct/total:.2%} ({correct}/{total})")

if position_total:
    print(f"\nPer-Position Accuracy:")
    for pos in sorted(position_total.keys()):
        pos_acc = position_correct[pos] / position_total[pos]
        print(f"  Position {pos}: {pos_acc:.2%} ({position_correct[pos]}/{position_total[pos]})")


In [ ]:
import pandas as pd

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Filter for only incorrect responses
correct_ans = df[df['correct'] == True]

print(f"Total successes: {len(correct_ans)}")
print("\n--- Examples of correct Answers ---")
# Display the first 20 failures
# We use .head(20) to see enough variety
display(correct_ans[['queried_object', 'expected', 'response', 'position']].head(20))
pd.set_option('display.max_colwidth', None)

failed = df[df['correct'] == False]

print(f"Total Failures: {len(failed)}")
print("\n--- Examples of incorrect Answers ---")
# Display the first 20 failures
# We use .head(20) to see enough variety
display(failed[['queried_object', 'expected', 'response', 'position','full_query']].head(20))